## MLflow Tracing with Gemini 2.0 Flash

In [19]:
import mlflow
import google.generativeai as gemini
from dotenv import load_dotenv
import os

load_dotenv()

mlflow.set_experiment("gemini-tracing")

mlflow.gemini.autolog()

## Basic Request

In [4]:
gemini.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = gemini.GenerativeModel("gemini-2.0-flash-exp")
response = model.generate_content("What is MLflow?")
print(response)


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "MLflow is an open-source platform to manage the machine learning lifecycle. It's designed to address the challenges of developing, deploying, and managing machine learning models, including tracking experiments, packaging code for reproducibility, and deploying models to various environments.\n\nThink of it as a central hub that helps data scientists and machine learning engineers collaborate and streamline their workflows.\n\nHere's a breakdown of its key features and functionalities:\n\n**Key Components of MLflow:**\n\n* **MLflow Tracking:**\n    * **Purpose:** Records and queries experiments:\n        * **Parameters:** Hyperparameters used in the model training.\n        * **Metrics:** Evaluation scores like accuracy, loss, F1 score.\n        * **Artifact

# Tool Calling Example

## Using the Gemini SDK

In [18]:
# Define the tool/function that Gemini can call
def calculate_tip(bill_amount: float, tip_percentage: float) -> float:
    """
    Calculate the tip amount based on the bill amount and tip percentage.

    Args:
        bill_amount (float): The total bill amount.
        tip_percentage (float): The percentage of the bill to be given as a tip.

    Returns:
        float: The calculated tip amount.
    """
    return bill_amount * (tip_percentage / 100)

model = gemini.GenerativeModel("gemini-2.0-flash-exp", tools=[calculate_tip])
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message("What is the tip for a $187.32 bill with a 22% tip?")

response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The tip for a $187.32 bill with a 22% tip is $41.21.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }
          ],
          "avg_logprobs": -0.0004010262366916452

## Using the OpenAI SDK

In [14]:
from openai import OpenAI

mlflow.openai.autolog()

client = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Define the function schema in OpenAI format
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate_tip",
            "description": "Calculate the tip amount based on the bill amount and tip percentage",
            "parameters": {
                "type": "object",
                "properties": {
                    "bill_amount": {
                        "type": "number",
                        "description": "The total bill amount"
                    },
                    "tip_percentage": {
                        "type": "number",
                        "description": "The percentage of the bill to be given as a tip"
                    }
                },
                "required": ["bill_amount", "tip_percentage"]
            }
        }
    }
]

def calculate_tip(bill_amount: float, tip_percentage: float) -> float:
    return bill_amount * (tip_percentage / 100)

# Make the request
response = client.chat.completions.create(
    model="gemini-2.0-flash-exp",
    messages=[
        {"role": "user", "content": "What is the tip for a $187.32 bill with a 22% tip?"}
    ],
    tools=tools,
    tool_choice="required",
)

# Handle the function call
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    function_args = json.loads(tool_call.function.arguments)
    
    # Execute the function
    result = calculate_tip(
        bill_amount=function_args["bill_amount"],
        tip_percentage=function_args["tip_percentage"]
    )
    
    print(f"The tip amount is ${result:.2f}")

The tip amount is $41.21
